# Exploratory data analysis - First understanding of the data

## Consume the pre-calculated metadata

In [2]:
!pip install pandas-profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.6/262.6 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 70.7 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 696.1/696.1 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.6/812.6 kB 36.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.1 MB/s eta 0:00:00:00:0100:01
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27096 sha256=f261b05e9d43936b05d65066ce33a777f0205d07ad24123d8e8017b1cc2914b8
  Stored in directory: /home/ydata/.cache/pip/wheels/23/14/6e/4be5bfeeb027f4939a01764b48edd5996acf574b0913fe5243
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.

In [31]:
import os

from pickle import load
import pandas as pd

from ydata.metadata import Metadata

def load_sets(path: str):
    with open(path, 'rb') as f:
        return load(f)

In [32]:
try:
    set_path = os.environ['SET_PATH']
except:
    set_path = 'train.pkl'
    
data, y = load_sets(set_path)

In [33]:
try:
    data = data.drop('Unnamed: 0', axis=1)
except:
    "No unnamed"

In [34]:
meta = Metadata.load('metadata.pkl')
print(meta)

Metadata Summary 
 
Dataset type: TABULAR
Dataset attributes: 
Number of columns: 32
% of duplicate rows: 26
Target column: 

Column detail: 
                            Column    Data type Variable type
0                            hotel  categorical        string
1                      is_canceled  categorical           int
2                        lead_time  categorical           int
3                arrival_date_year  categorical           int
4               arrival_date_month  categorical        string
5         arrival_date_week_number  categorical           int
6        arrival_date_day_of_month  categorical           int
7          stays_in_weekend_nights  categorical           int
8             stays_in_week_nights  categorical           int
9                           adults  categorical           int
10                        children  categorical         float
11                          babies  categorical           int
12                            meal  categorical     

## Generating the full data profile

In [7]:
try:
    data_split=os.environ['DATA_SPLIT']
except:
    data_split='train'

In [8]:
from pandas_profiling import ProfileReport

print(f'Profile Name: {data_split}_profile')
profile = ProfileReport(df=data, title='Hotel bookings demand')
profile.config.html.navbar_show = False

profile.to_file(f'{data_split}_profile.html')

Profile Name: train_profile


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/opt/conda/lib/python3.8/site-packages/scipy/stats/stats.py:4594: RuntimeWarning: overflow encountered in long_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Add here the html artifact to be generated

In [37]:
ratio_labels = pd.DataFrame(y.value_counts(normalize=True))

In [39]:
ratio_labels

,is_canceled
0,0.627481
1,0.372519


In [43]:
import json

metadata = {
    'outputs' : [
        {
      'type': 'table',
      'storage': 'inline',
      'format': 'csv',
      'header': list(ratio_labels.columns),
      'source': ratio_labels.to_csv(header=False, index=True)
    },
        {
      'type': 'web-app',
      'storage': 'inline',
      'source': profile.to_html(),
    }
    ]
  }

with open('mlpipeline-ui-metadata.json', 'w') as metadata_file:
    json.dump(metadata, metadata_file)